In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# keras
import keras

# matplotlib
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

import os

In [3]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv('spam_or_not_spam.csv')
df.head()

,email,label
0,mike bostock said received from trackingNUMBE...,0
1,no i was just a little confused because i m r...,0
2,this is just an semi educated guess if i m wro...,0
3,jm URL justin mason writes except for NUMBER t...,0
4,i just picked up razor sdk NUMBER NUMBER and N...,0


In [6]:
df.label.value_counts()

0    1000
1     500
Name: label, dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   email   1499 non-null   object
 1   label   1500 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 23.6+ KB


In [8]:
#Afairoume th grammi pou einai null to email

df.dropna(inplace=True)

In [9]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

def tokenizer_sequences(num_words, X):
    
   
    tokenizer = Tokenizer(num_words=num_words)
    
    sequences = tokenizer.texts_to_sequences(X)
    
    return tokenizer, sequences

In [11]:
max_words = 10000 

maxlen = 300
# maxlen is the dimension that each email will have a fixed word sequence, in this case each email will be of a 1-d tensor (300,).

In [12]:
tokenizer, sequences = tokenizer_sequences(max_words, df.email.copy())

In [13]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))


X = pad_sequences(sequences, maxlen=maxlen)

y = df.label.copy()

print('Shape of data tensor:', X.shape)
print('Shape of label tensor:', y.shape)

Found 22120 unique tokens.
Shape of data tensor: (1499, 300)
Shape of label tensor: (1499,)


In [14]:
max_words = len(tokenizer.word_index) + 1 # 33672 + 1

embedding_dim = 100 

In [15]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding

model = Sequential()


model.add(Embedding(max_words, embedding_dim, input_length=maxlen)) 

model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 300, 100)          2212100   
_________________________________________________________________
flatten_1 (Flatten)          (None, 30000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 30001     
Total params: 2,242,101
Trainable params: 2,242,101
Non-trainable params: 0
_________________________________________________________________


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [17]:
X_train.shape, y_train.shape

((1199, 300), (1199,))

In [18]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

history = model.fit(X_train, y_train, epochs=3, batch_size=32, validation_split=0.2)

Train on 959 samples, validate on 240 samples
Epoch 1/3
959/959 [==============================] - 4s 4ms/step - loss: 0.5113 - acc: 0.7779 - val_loss: 0.4608 - val_acc: 0.8250
Epoch 2/3
959/959 [==============================] - 2s 3ms/step - loss: 0.2602 - acc: 0.9385 - val_loss: 0.2431 - val_acc: 0.9167
Epoch 3/3
959/959 [==============================] - 3s 3ms/step - loss: 0.1125 - acc: 0.9698 - val_loss: 0.1400 - val_acc: 0.9458


In [19]:
model.evaluate(X_test, y_test)
# loss value & acc metrics

300/300 [==============================] - 0s 157us/step


[0.11522872239351273, 0.95333331823349]

In [20]:
model.layers[0].get_weights()[0].shape
# (vocabulary len x dimension of word vector) = word vector

(22121, 100)

In [21]:
model2 = Sequential()

model2.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model2.add(Flatten())
model2.add(Dense(32, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))
model2.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 300, 100)          2212100   
_________________________________________________________________
flatten_2 (Flatten)          (None, 30000)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                960032    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 3,172,165
Trainable params: 3,172,165
Non-trainable params: 0
_________________________________________________________________


In [22]:
model2.layers[0].set_weights(model.layers[0].get_weights()) # load
model2.layers[0].trainable = False # freeze

In [23]:
model2.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

history2 = model2.fit(X_train, y_train, epochs=2, batch_size=32, validation_split=0.2)

Train on 959 samples, validate on 240 samples
Epoch 1/2
959/959 [==============================] - 2s 2ms/step - loss: 0.1176 - acc: 0.9604 - val_loss: 0.1111 - val_acc: 0.9583
Epoch 2/2
959/959 [==============================] - 1s 873us/step - loss: 0.0288 - acc: 0.9896 - val_loss: 0.0912 - val_acc: 0.9667


In [27]:
from sklearn.metrics import precision_score, recall_score,f1_score

y_test_pred = np.where(model2.predict(X_test) > .5, 1, 0).reshape(1, -1)[0]

In [28]:
print("Precision: {:.2f}%".format(100 * precision_score(y_test, y_test_pred)))
print("Recall: {:.2f}%".format(100 * recall_score(y_test, y_test_pred)))

Precision: 97.87%
Recall: 92.00%


In [29]:
# predict the test score
model2.evaluate(X_test, y_test)
# loss value & acc metrics

300/300 [==============================] - 0s 282us/step


[0.05351869293799003, 0.9666666388511658]

In [33]:
f1_score= 2 * (97.87*92.00)/(97.87*92.00)

In [41]:
from sklearn.metrics import classification_report
y_test_pred = np.where(model2.predict(X_test) > .5, 1, 0).reshape(1, -1)[0]

ValueError: Number of classes, 2, does not match size of target_names, 3. Try specifying the labels parameter